In [30]:
import pandas as pd
import numpy as np
import requests
import itertools
from google.cloud import bigquery

In [31]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [32]:
#Authentication - without user
auth_url = 'https://accounts.spotify.com/api/token'
cid = 'cf7f9afe97a84fa0843d6ff46c0d1abe'
csecret = '94f332ce5752459581cb462a0c246c6c'
data = {
    'grant_type': 'client_credentials',
    'client_id': cid,
    'client_secret': csecret,
}
auth_response = requests.post(auth_url, data=data)
access_token = auth_response.json().get('access_token')

In [33]:
base_url = 'https://api.spotify.com/v1/'
headers = {
    'Authorization': 'Bearer {}'.format(access_token)
}

In [34]:
podcast = 'data+hackers'
tipo = 'show'
pais = 'BR'
limite = 50
offset = 0

shows_endpoint = 'search?q='+ podcast + '&type=' + tipo + '&market=' + pais + '&limit=' + str(limite) + '&offset=' + str(offset)
shows_url = base_url + shows_endpoint
shows_response = requests.get(shows_url,headers=headers).json()

In [35]:
shows = shows_response['shows']

In [36]:
df_raw = pd.DataFrame.from_records(shows)

In [37]:
for i in range(0,50):
  if df_raw.iloc[i,1] is None:
    df_raw.iloc[i,1] = 'none'

In [38]:
shows_items = df_raw['items'].to_list()

In [39]:
df_shows = pd.DataFrame(columns=['id', 'name', 'description', 'total_episodes'])

In [40]:
shows_dados = []
for i in range(len(shows_items)):
  if shows_items[i] != 'none':
   linha = {'id':shows_items[i]['id'], 'name':shows_items[i]['name'], 'description':shows_items[i]['description'], 'total_episodes':shows_items[i]['total_episodes']}
  else: linha = {'id':'none', 'name':'none', 'description':'none', 'total_episodes':0}
  shows_dados.append(linha)


In [41]:
df_shows = pd.DataFrame.from_records(shows_dados)

In [42]:
df_shows

,id,name,description,total_episodes
0,1oMIHOXsrLFENAeM743g93,Data Hackers,O podcast oficial do Data Hackers: a maior com...,107
1,1nfBfEtCmuvJkPQgR3bS5a,Data Career Podcast,The Data Career Podcast: helping you break int...,99
2,7r3ijPPBDnBAlN7iv8m8M2,Hidden Files,It aims at creating awareness around hackers s...,100
3,7kU6yrZ33zDGmuCLXepEPK,It's All About Data/Internet These Days. Hacke...,Data/Internet security you name it I'm interes...,1
4,02yJXEJAJiQ0Vm2AO9Xj6X,DataFramed,"Welcome to DataFramed, a weekly podcast explor...",198
5,3YRafqb2OGxfXgoIPfRbe2,Exploit Brokers - Tech and Hacking News Commen...,Exploit brokers is dedicated to tech and hacki...,19
6,21zZfOy7VCSIIWlJ64DElv,Hacked,"Strange tales of hacking, tech, internet grift...",85
7,none,none,none,0
8,1BZN7H3ikovSejhwQTzNm4,Data Skeptic,Data Skeptic is your source for a perspective ...,518
9,5wcjfEn1GxY8fOhjNYs0FV,"The Elite HRV Podcast: Heart Rate Variability,...",Uncommon methods for maximizing health and per...,63


In [51]:
client = bigquery.Client(project='gb-test-415817')

table = 'spotify.dh_pesquisa'

job = client.load_table_from_dataframe(df_shows, table).result()

In [43]:
id_dh = df_shows.iloc[0,0]
eps_dh = df_shows.iloc[0,3]

In [44]:
dh_dados_raw = []

for i in [0,50,100]:
  dh_endpoint = 'shows/'+ id_dh + '/episodes?market=' + pais + '&limit=50&offset=' + str(i)
  dh_url = base_url + dh_endpoint
  dh_response = requests.get(dh_url,headers=headers).json().get('items')
  dh_dados_raw.append(dh_response)

In [45]:
dh_dados_raw = list(itertools.chain.from_iterable(dh_dados_raw))
len(dh_dados_raw)

107

In [46]:
dh_dados = []
for i in range(len(dh_dados_raw)):
  linha = {'id':dh_dados_raw[i]['id'],
           'name':dh_dados_raw[i]['name'],
           'description':dh_dados_raw[i]['description'],
           'release_date':dh_dados_raw[i]['release_date'],
           'duration_ms':dh_dados_raw[i]['duration_ms'],
           'language':dh_dados_raw[i]['language'],
           'explicit':dh_dados_raw[i]['explicit'],
           'type':dh_dados_raw[i]['type']}
  dh_dados.append(linha)

In [47]:
dh_df = pd.DataFrame.from_records(dh_dados)

In [48]:
dh_df

,id,name,description,release_date,duration_ms,language,explicit,type
0,2DulxkkUl7ZeXZxjHuYCHg,CEO da Nvidia diz que crianças não devem apren...,"Está no ar, o Data Hackers News !! Os assuntos...",2024-02-28,820741,pt-BR,False,episode
1,1naU4YEKsRAiicqxA2CfAy,"Salários, stocks options e compensações em tec...",Quais são as formas de ganhar dinheiro em tecn...,2024-02-23,3816228,pt-BR,False,episode
2,4jeKPdEmUOJJ44QQYFh8cu,"OpenAI anuncia Sora, a IA de geração de vídeos...","Está no ar, o Data Hackers News !! Os assuntos...",2024-02-21,862876,pt-BR,False,episode
3,1YTYcE7g9dScXCOvQXMO4E,Neuralink implanta chip em cerebro humano pela...,"Está no ar, o Data Hackers News !! Os assuntos...",2024-02-07,919797,pt-BR,False,episode
4,2XvbpGsehEjIuvAmhszQAW,Data Hackers News #14 - Meta anuncia criação d...,"Está no ar, o Data Hackers News !! Os assuntos...",2024-02-01,1072822,pt-BR,False,episode
...,...,...,...,...,...,...,...,...
102,7Bg1wXMnQbJMeoJ1RjEXvm,Como criar um time de Data Science - Episódio ...,"Fala, Data Hackers. Sejam muito bem-vindos a m...",2019-01-06,3522900,pt-BR,False,episode
103,7jV5hVnSDQtRGpO3kPdLxV,O que um Engenheiro de Dados faz? - Episódio 0...,"Olá, Data Hackers. Sejam muito bem-vindos a ma...",2018-12-09,3641940,pt-BR,False,episode
104,11mVafwx00vWHOLf5kiYLY,Data-Driven Products: por dentro da área de pr...,Sejam muito bem-vindos a mais um episódio do s...,2018-11-19,2963328,pt-BR,False,episode
105,7kfQHi3sxNH3RhZXZL8i9X,Como é o dia a dia de um Cientista de Dados - ...,"Olá, Data Hacker! Bem-vindo a mais um episódio...",2018-10-14,3589826,pt-BR,False,episode


In [52]:
client = bigquery.Client(project='gb-test-415817')

table = 'spotify.dh_episodios'

job = client.load_table_from_dataframe(dh_df, table).result()